# Specify Path of MINE_net to load

In [1]:
import os
from os import path as p

Use dropdown for the following instead
- For consistency with the naming convention, change 'Mixed Gaussian' to 'Mixed_Gaussian'
- For consistency with the parameter name in the code, population_size to n_samples.

In [2]:
import ipywidgets as widgets
from IPython.display import display, clear_output

In [3]:
proj_path = p.dirname(p.abspath('MINE-Entropy'))
exp_path = p.join(proj_path, "results")
print(exp_path)

C:\Users\Michael\Documents\GitHub\michael\MINE-Entropy\results


In [4]:
batch_size=128
population_size=2048
sub_path = p.join(exp_path, "pop={}_batch={}".format(population_size, batch_size))
print(sub_path)

C:\Users\Michael\Documents\GitHub\michael\MINE-Entropy\results\pop=2048_batch=128


In [5]:
datasets = ["Mixed Gaussian", "Gaussian", "Mixed Uniform"]
select_data = widgets.Dropdown(options=datasets, description="Select data: ")
display(select_data)

Dropdown(description='Select data: ', options=('Mixed Gaussian', 'Gaussian', 'Mixed Uniform'), value='Mixed Ga…

In [6]:
data_name=select_data.value

vary_name="rho1"
vary_val=0.9
data_path = p.join(sub_path, "{}_{}={}".format(data_name, vary_name, vary_val))
print("Data path is:")
print(data_path)

Data path is:
C:\Users\Michael\Documents\GitHub\michael\MINE-Entropy\results\pop=2048_batch=128\Mixed Gaussian_rho1=0.9


In [7]:
models = ["MINE_direct", "MINE_entropy", "MINE_multi_task"]
select_model = widgets.Dropdown(options=models, description="Select model: ")
display(select_model)

Dropdown(description='Select model: ', options=('MINE_direct', 'MINE_entropy', 'MINE_multi_task'), value='MINE…

In [8]:
model_name=select_model.value
model_path = p.join(data_path, model_name)
print("Model path is:")
print(model_path)

Model path is:
C:\Users\Michael\Documents\GitHub\michael\MINE-Entropy\results\pop=2048_batch=128\Mixed Gaussian_rho1=0.9\MINE_direct


Output directory to store plots

In [9]:
out_path = p.join(proj_path,'Output')
os.makedirs(p.join(proj_path,'Output'),exist_ok=True)

# Import minee and needed packages

In [10]:
import minee 
import torch

In [11]:
from minee import settings
MINE_dir = settings.model[model_name]['model']

# Load Mine_net

In [12]:
iter_num = 78125
mine_file = p.join(model_path, "checkpoint_iter={}.pt".format(iter_num))
print(mine_file)
MINE_dir.mine_net.load_state_dict(torch.load(mine_file))

C:\Users\Michael\Documents\GitHub\michael\MINE-Entropy\results\pop=2048_batch=128\Mixed Gaussian_rho1=0.9\MINE_direct\checkpoint_iter=78125.pt


# Load Train and valid mi_lb curve

In [13]:
import numpy as np
MINE_dir.avg_train_mi_lb = np.loadtxt(p.join(model_path, "avg_train_mi_lb.txt"))
MINE_dir.avg_valid_mi_lb = np.loadtxt(p.join(model_path, "avg_valid_mi_lb.txt"))
# MINE_dir.avg_train_loss = np.loadtxt(p.join(model_path, "avg_train_loss.txt"))
# MINE_dir.avg_test_loss = np.loadtxt(p.join(model_path, "avg_test_loss.txt"))

# Generate Data

Allow the input of a random seed. Can use 0 as the default for our result, so there is no need to store the random seed.

In [14]:
#Default value of datasets are shown
#Data can also be user-specified
def generate_dataset(selected_data):
    if (selected_data == 'Mixed Gaussian'):
        dataset = minee.data.mix_gaussian.MixedGaussian(n_samples=400, mean1=0, mean2=0, rho1=0.9, rho2=-0.9, mix=0.5, theta=0)
    elif (selected_data == 'Gaussian'):
        dataset = minee.data.gaussian.Gaussian(n_samples=200, mean1=0, mean2=1, rho=0.8)
    elif (selected_data == 'Mixed Uniform'):
        dataset = minee.data.mix_uniform.MixedUniform(mix=0.5, width_a=10, width_b=10, n_samples=200)
    return dataset
gaus = generate_dataset(data_name)
X = gaus.data

# Generate figure

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
def train_mine(X):
    MINE_dir.X_train, MINE_dir.X_test = train_test_split(X, test_size=0.35, random_state=0)
    MINE_dir.ground_truth = gaus.ground_truth
    mi_lb = MINE_dir.forward_pass(MINE_dir.X_test).item()
    MINE_dir.savefig(X, mi_lb, suffix="_test2")

Generate pgf plot instead of png for inclusion in LaTeX. Also present the plots in jupyter notebook.
See https://matplotlib.org/users/pgf.html

In [17]:
import matplotlib as mpl
import matplotlib.pyplot as plt
pgf_with_rc_fonts = {
    "pgf.rcfonts" : False,
    "axes.unicode_minus" : False,
    "pgf.texsystem": "pdflatex"
}
mpl.rcParams.update(pgf_with_rc_fonts)

In [18]:
def plot_data(X, data_name):
    #show plot
    data_plot = plt.figure()
    plt.scatter(X[:,1], X[:,0], color='red', marker='o')
    plt.title('Scatter Plot of %s Data' % (data_name))
    plt.show()
    #save figure
    data_plot.savefig(p.join(out_path,'data.pdf'),bbox_inches='tight') # For preview
    data_plot.savefig(p.join(out_path,'data.pgf')) # For inclusion in latex

In [19]:
# Define output box
output_plot = widgets.Output()

# Function to train the algorithm and plot the results
def train_and_plot():
    output_plot.clear_output()
    #Update data and model
    gaus = generate_dataset(select_data.value)
    X = gaus.data
    MINE_dir = settings.model[select_model.value]['model']
    with output_plot:
        print("Running...")
    train_mine(X)
    output_plot.clear_output()
    with output_plot:
        plot_data(X, select_data.value)
        
#Update data or model according to user selection and rerun
def changed_param(b):
    train_and_plot()
    
# Observe if data selection is changed
select_data.observe(changed_param, names="value")
select_model.observe(changed_param, names="value")

In [20]:
# Display dropdown to allow user to select another data or model and rerun
display(select_data)
display(select_model)

# Output box for data plot
display(output_plot)

# Run according to parameters defined before
train_and_plot()

Dropdown(description='Select data: ', options=('Mixed Gaussian', 'Gaussian', 'Mixed Uniform'), value='Mixed Ga…

Dropdown(description='Select model: ', options=('MINE_direct', 'MINE_entropy', 'MINE_multi_task'), value='MINE…

Output()

# Explore other data

Implementation is not yet finished

In [21]:
select_model_other = widgets.Dropdown(options=models, description="Select model: ")
display(select_model_other)

Dropdown(description='Select model: ', options=('MINE_direct', 'MINE_entropy', 'MINE_multi_task'), value='MINE…

In [22]:
# MINE_dir = settings.model[select_model_other.value]['model']
# # X = 
# # data_name = 
# train_mine(X)
# plot_data(X, data_name)